In [2]:
from google.colab import files
import zipfile
import os

uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('./')
        print(f"✅ Extracted: {filename}")

os.listdir('./')


Saving waste_classifier.zip to waste_classifier.zip
✅ Extracted: waste_classifier.zip


['.config', 'waste_classifier.zip', 'waste_classifier', 'sample_data']

In [5]:
import cv2
import numpy as np
import tensorflow as tf

model_path = '/content/waste_classifier/waste_classifier.keras'
model = tf.keras.models.load_model(model_path)

def predict_image(frame):
    img = cv2.resize(frame, (150, 150))
    img_array = img / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    return predicted_class

class_labels = {0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'plastic'}

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape
    roi = frame[height//4:height*3//4, width//4:width*3//4]

    predicted_class = predict_image(roi)
    waste_type = class_labels.get(predicted_class, 'Unknown')

    cv2.rectangle(frame, (width//4, height//4), (width*3//4, height*3//4), (255, 0, 0), 2)
    cv2.putText(frame, f'Waste Type: {waste_type}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Waste Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [6]:
from google.colab import output
from google.colab import files
from IPython.display import Image
import cv2
import numpy as np
import tensorflow as tf

from js import navigator

def take_photo(filename='photo.jpg', quality=0.8):
    js_code = """
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
      document.body.appendChild(div);
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      const video = document.createElement('video');
      video.srcObject = stream;
      await video.play();
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      await new Promise((resolve) => capture.onclick = resolve);
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      div.remove();
      const dataUrl = canvas.toDataURL('image/jpeg', quality);
      const response = await fetch(dataUrl);
      const blob = await response.blob();
      const arrayBuffer = await blob.arrayBuffer();
      return new Uint8Array(arrayBuffer);
    }
    """
    display(output.Javascript(js_code))
    data = output.eval_js('takePhoto({})'.format(quality))
    with open(filename, 'wb') as f:
        f.write(data.tobytes())
    return filename

filename = take_photo()
print("✅ Captured:", filename)
img = cv2.imread(filename)
cv2.imshow("Captured Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'js'